In [ ]:
!pip install python_speech_features
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np

In [ ]:
def getNeighbors(train_Set, instance, j):
    distances = []
    for x in range (len(train_Set)):
        dist = distance(train_Set[x], instance, j )+ distance(instance, train_Set[x], j)
        distances.append((train_Set[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(j):
        neighbors.append(distances[x][0])
    return neighbors

In [ ]:
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [ ]:
def getAccuracy(test_Set, predictions):
    correct = 0 
    for x in range (len(test_Set)):
        if test_Set[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(test_Set)

In [ ]:
directory = "/content/drive/MyDrive/genres/"
f= open("my.dat" ,'wb')
i=0

for folder in os.listdir(directory):
    i+=1
    if i==11 :
        break 	
    for file in os.listdir(directory+folder):	
        (rate,sig) = wav.read(directory+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covar = np.cov(np.matrix.transpose(mfcc_feat))
        mean_mat = mfcc_feat.mean(0)
        feat = (mean_mat , covar , i)
        pickle.dump(feat , f)

f.close()

In [ ]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break	

    for x in range(len(dataset)):
        if random.random() <split :			
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])	

train_Set = []
test_Set = []
loadDataset("my.dat" , 0.66, train_Set, test_Set)

In [ ]:
def distance(instance1 , instance2 , j ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= j
    return distance

In [ ]:
length = len(test_Set)
predictions = []
for x in range (length):
    predictions.append(nearestClass(getNeighbors(train_Set ,test_Set[x] , 5))) 

accuracy1 = getAccuracy(test_Set , predictions)
print(accuracy1)

In [ ]:
test_dir = "/content/drive/MyDrive/test set/"
# test_file = test_dir + "1999.wav"
test_file = test_dir + "survesdre.wav"
# test_file = test_dir + "purify.wav"

In [ ]:
(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covar = np.cov(np.matrix.transpose(mfcc_feat))
mean_mat = mfcc_feat.mean(0)
feat = (mean_mat, covar, i)

In [ ]:
from collections import defaultdict
results = defaultdict(int)

directory = "/content/drive/MyDrive/genres/"
i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [ ]:
pred = nearestClass(getNeighbors(dataset, feat, 5))
print(results[pred])